# 0.0 Problema

**Objetivo**: Se aproximar o máximo possivel do gráfico feito pela Finacial Times.



[<img src="https://www.ft.com/__origami/service/image/v2/images/raw/https%3A%2F%2Fd6c748xw2pzm8.cloudfront.net%2Fprod%2F2cc43cb0-d713-11ea-8827-2bf5fdc831fb-fullwidth.png?dpr=1&fit=scale-down&quality=highest&source=next&width=1260">](https://www.ft.com/content/272354f2-f970-4ae4-a8ae-848c4baf8f4a)

Para isto, é interessante quebrar o escopo em vários escopos:
- [ ] Definir o que vai ser Estudado
- [ ] Importar Bibliotecas
- [ ] Limpar os Dados
- [ ] Plotar o Gráfico



Para esse notebook, vamos utilizar a curva de Retail, dos estados do Brasil.

Fizemos então as seguintes definições:

  - Tratar Apenas dos estados Brasileiros
  - Tratar Apenas do campo *RETAIL* (ida ao varejo)
  - Gerar um gráfico mais próximo do FT.


Analisando a foto do FT, temos alguns escopos dentro do gráfico:
  - Gerar um gráfico mais próximo do FT.
    - É um gráfico agrupado pela data *(eixo X)* pelo RETAIL *(eixo Y)*
    - Ter um Estado em Destaque e o resto em fundo
    - Ter o Valor Minimo da Curva
    - Ter o Valor Maximo da Curva
    - Ter a variação do Mínimo com o último valor

Dado isso, é possivel que teremos algumas funções

  - Agrupar
  - Minimo
  - Ultimo
  - Variacao


# 1.0 Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects


# 2.0 Importando Dataset

In [ ]:
link = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"


df = pd.read_csv(link)

In [ ]:
df.head(2)

# 3.0 Limpando os Dados

## 3.1 Limpando os Nomes

In [ ]:
## Escolhendo Apenas a Região do Brasil
df = df.loc[df.country_region == "Brazil",:].iloc[:,[2,3,7,8]]

## Ajustando os Nomes das colunas
df.columns = ["estado","cidade","data","ida_ao_varejo"]

##
df.estado.replace('Federal District','Distrito Federal', inplace=True)

In [ ]:
df.head(2)

## 3.2 Transformando em Série Temporal

In [ ]:
 ## Transformando a data (tipo string) para datetime (tipo datetime)
 df.data = pd.to_datetime(df.data)

 ## colocando a data no lugar do index
 df.index = df.data

 ## removendo a coluna date
 df.drop(labels="data",axis=1,inplace=True)


 # removendo State of
 df.estado = df.estado.str.replace('State of ','')

In [ ]:
df.head(2)

# 4.0 Função Groupby


In [ ]:
## Criar uma função para agrupar o Dataset
## E ele já irá fazer a média e destacar do grupo
def agrupar(df,agruparPor,estados = ''):
  if(estados):
    return df.groupby(by=agruparPor).mean().unstack()['ida_ao_varejo'][estados].rolling(window=7).mean()
  else:
    estados = df.estado.unique().tolist()
    estados.pop(0)
    return df.groupby(by=agruparPor).mean().unstack()['ida_ao_varejo'][estados].rolling(window=7).mean()

# 5.0 Plotagem do Gráfico

## 5.1 Função Mínimo, Último e sua Variação em Porcentagem

In [ ]:
## Vamos pegar o Index do valor minimo
## E o valor do minimo
## Retornar o index e o Ultimo Valor
def getMinimo(serie):
    index = serie.idxmin()
    min = serie.min()
    return index,min


In [ ]:
## Vamos pegar o Index do ultimo valor
## E o valor do último
## Retornar o index e o Ultimo Valor
def getUltimo(serie):
  index = serie.tail(1).index.item()
  ultimo = serie.iloc[-1]
  return index,ultimo

In [ ]:
## Vamos dividir o ultimo pelo minimo, multiplicar por 100 e tirar 100
## depois usamos o abs para transformar em positivo e passamos o str para
## transformar em String, depois concatemos com % e retornamos
def getVariacao(minimo,ultimo):
  return '{0:.2f}'.format(abs(((ultimo/minimo)*100)-100)) + '%'

## 5.2 Plotagem 1 Estado

Para testar vamos iniciar apenas com 1 estado.

In [ ]:
## Pegando os Valores
# DF RN
df_RN = agrupar(df,[df.index,df.estado],'Rio Grande do Norte')
# DF COM TODOS ESTADOS
df_todos_estados = agrupar(df,[df.index,df.estado])

#Index Min,ultimo e valores e sua variacao do RN
indexMin_RN, min_RN = getMinimo(df_RN.dropna())
indexUltimo_RN, ultimo_RN = getUltimo(df_RN.dropna())
variacao_RN = getVariacao(min_RN,ultimo_RN)

In [ ]:
data_RN = indexMin_RN.strftime("%d/%m/%Y")

In [ ]:
## Em verde é as modificações desse código 
## zorder o quanto pra cima o bagulho ta

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,8))


## Plot do RN
df_RN.plot(linewidth=6,zorder=3,color='#35d0ba', alpha=1,ax=ax)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles,labels, ncol=2)

ax.set_xticklabels(["","Ago","","","","","","Set"])
ax.set_xlabel("")

# Bolinha no Minimo
ax.scatter(indexMin_RN,min_RN,marker='o',color='#ff9234',linewidth=6,zorder=4)
ax.annotate('{0:.2f}'.format(min_RN),xy=(indexMin_RN,min_RN+1),fontsize=12,color='#ff9234',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])
ax.annotate(data_RN,xy=(indexMin_RN,min_RN-5),fontsize=12,color='#ff9234',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])


# Bolinha no ultimo
ax.scatter(indexUltimo_RN,ultimo_RN,marker='o',color='#d92027',linewidth=6,zorder=4)
ax.annotate('{0:.2f}'.format(ultimo_RN),xy=(indexUltimo_RN,ultimo_RN+2),fontsize=12,color='#ff9234',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])

# Tracinhos do Capeta
# Tranformação do Timestamp em Period
ax.plot([pd.Period(indexMin_RN.ctime(), 'D'), pd.Period(indexUltimo_RN.ctime(), 'D')], [min_RN, min_RN], color="#ff9234", linestyle="--", linewidth=2)
ax.plot([pd.Period(indexUltimo_RN.ctime(), 'D'), pd.Period(indexUltimo_RN.ctime(), 'D')], [min_RN, ultimo_RN], color="#d92027", linestyle="--", linewidth=2)

# Porcentagem Variação
ax.annotate(variacao_RN,xy=(indexUltimo_RN,-50),fontsize=14,color='#d92027',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])

## Fundo do Plot
fig.patch.set_facecolor('#fff1e5')
ax.set_facecolor('#fff1e5')

## Plot dos Outros
df_todos_estados.plot(legend=False,color="grey",linewidth=1, alpha=0.4, ax=ax)


## Alterando o nome
ax.set_title("Recuperação do RN",fontsize=14,color='black',weight='bold',ha='center')



plt.show()

## 5.3 Plotagem para Estados Selecionados

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(32,8))
#Colocando cor na area de algo


estados = ["Pará", "Piauí", "Rio Grande do Norte","São Paulo","Santa Catarina"]

df_todos_estados = agrupar(df,[df.index,df.estado])

for i,estado in enumerate(estados):

  #Agrupando por Index e Estado
  df_destaque = agrupar(df,[df.index,df.estado],estado)

  ##Colocando os outros Estados em Cinza
  df_todos_estados.plot(legend=False,color="grey",linewidth=1, alpha=0.4, ax=ax[i])


  ## Colocando o Estado em Destaque
  df_destaque.plot(linewidth=6,zorder=3,color='#35d0ba', alpha=1,ax=ax[i])


  # Pegar os Mínimos
  indexMin, min = getMinimo(df_destaque.dropna())
  indexUltimo, ultimo = getUltimo(df_destaque.dropna())
  variacao = getVariacao(min,ultimo)

  # Bolinha no Minimo
  ax[i].scatter(indexMin,min,marker='o',color='#ff9234',linewidth=6,zorder=4)
  ax[i].annotate('{0:.2f}'.format(min),xy=(indexMin,min+1),fontsize=14,color='black',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])
  

  # Bolinha no ultimo
  ax[i].scatter(indexUltimo,ultimo,marker='o',color='#ff9234',linewidth=6,zorder=4)
  ax[i].annotate('{0:.2f}'.format(ultimo),xy=(indexUltimo,ultimo+2),fontsize=14,color='black',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])

  # Porcentagem Variação
  ax[i].annotate(variacao,xy=(indexUltimo,-50),fontsize=14,color='#d92027',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])
  
  # Tracinhos do Capeta
  # Tranformação do Timestamp em Periodo
  ax[i].plot([pd.Period(indexMin.ctime(), 'D'), pd.Period(indexUltimo.ctime(), 'D')], [min, min], color="#ff9234", linestyle="--", linewidth=2)
  ax[i].plot([pd.Period(indexUltimo.ctime(), 'D'), pd.Period(indexUltimo.ctime(), 'D')], [min, ultimo], color="#d92027", linestyle="--", linewidth=2)


  ## Fundo do Plot
  fig.patch.set_facecolor('#fff1e5')
  ax[i].set_facecolor('#fff1e5')
  fig.patch.set_visible('#fff1e5')
  # ax[i].spines['right'].set_visible(False)
  # ax[i].spines['left'].set_visible(False)
  # ax[i].spines['top'].set_visible(False)

  ax[i].set_title(estado,fontsize=14,color='black',weight='bold',ha='center')
  ax[i].xaxis.grid(False)
  ax[i].set_facecolor('#fff1e5')
  ax[i].set_xlabel("")
  ax[i].arrow(6, 7, -2.5, -2.5, head_width = 0.5, 
         head_length = 0.5, fc ='g', ec ='g') 
  ax[i].set_xticklabels(["","Ago","","","","","","Set"])
  ax[i].xaxis.set_tick_params(labelsize=12)
  ax[i].yaxis.set_tick_params(labelsize=12)
  if (i==0) or (i==4):
    ax[i].yaxis.tick_right()
  else:
    ax[i].set_yticklabels([])

## 6.0 Analise para o Medium

In [ ]:
Regioes = {
    'Acre':'Norte',
    'Amapá':'Norte',
    'Amazonas':'Norte',
    'Pará':'Norte',
    'Rondônia':'Norte',
    'Roraima':'Norte',
    'Tocantins': 'Norte',
    'Alagoas': 'Nordeste',
    'Bahia': 'Nordeste',
    'Ceará': 'Nordeste',
    'Maranhão': 'Nordeste',
    'Paraíba': 'Nordeste',
    'Pernambuco': 'Nordeste',
    'Piauí': 'Nordeste',
    'Rio Grande do Norte': 'Nordeste',
    'Sergipe': 'Nordeste',
    'Goiás': 'Centro-Oeste',
    'Distrito Federal': 'Centro-Oeste',
    'Mato Grosso': 'Centro-Oeste',
    'Mato Grosso do Sul': 'Centro-Oeste',
    'Espírito Santo': 'Sudeste',
    'Minas Gerais': 'Sudeste',
    'Rio de Janeiro': 'Sudeste',
    'São Paulo': 'Sudeste',
    'Paraná': 'Sul',
    'Santa Catarina': 'Sul',
    'Rio Grande do Sul': 'Sul'
}


df['regiao'] = df.estado.map(Regioes)

In [ ]:
sul = df.estado.where(df.regiao == 'Sul').unique().tolist()
sul.pop(0)

norte = df.estado.where(df.regiao == 'Norte').unique().tolist()
norte.pop(0)
norte.pop(0)
print(norte)

nordeste = df.estado.where(df.regiao == 'Nordeste').unique().tolist()
nordeste.pop(0)


centro_oeste = df.estado.where(df.regiao == 'Centro-Oeste').unique().tolist()
centro_oeste.pop(0)

sudeste = df.estado.where(df.regiao == 'Sudeste').unique().tolist()
sudeste.pop(0)

sul = df.estado.where(df.regiao == 'Sul').unique().tolist()
sul.pop(0)


In [ ]:
gp = agrupar(df,[df.index,df.estado])

for regiao in [norte,nordeste,centro_oeste,sudeste,sul]:
 print(gp[regiao].rolling(window=7).mean().dropna().max())

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(32,8))
#Colocando cor na area de algo


estados = ["Tocantins", "Goiás", "Espírito Santo","Minas Gerais","Rio Grande do Sul"]

df_todos_estados = agrupar(df,[df.index,df.estado])

for i,estado in enumerate(estados):

  #Agrupando por Index e Estado
  df_destaque = agrupar(df,[df.index,df.estado],estado)
  
  for regiao in [norte,nordeste,centro_oeste,sudeste,sul]:
    df_regiao = agrupar(df,[df.index,df.estado],regiao)
  ##Colocando os outros Estados (Por Regiao) em Cinza
  df_regiao.plot(legend=False,color="grey",linewidth=1, alpha=0.4, ax=ax[i])


  ## Colocando o Estado em Destaque
  df_destaque.plot(legend=True, linewidth=4,zorder=3,color='#35d0ba', alpha=1,ax=ax[i])


  # Pegar os Mínimos
  indexMin, min = getMinimo(df_destaque.dropna())
  indexUltimo, ultimo = getUltimo(df_destaque.dropna())
  variacao = getVariacao(min,ultimo)

  # Bolinha no Minimo
  ax[i].scatter(indexMin,min,marker='o',color='#ff9234',linewidth=6,zorder=4)
  ax[i].annotate('{0:.2f}'.format(min),xy=(indexMin,min+1),fontsize=14,color='black',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])
  

  # Bolinha no ultimo
  ax[i].scatter(indexUltimo,ultimo,marker='o',color='#ff9234',linewidth=6,zorder=4)
  ax[i].annotate('{0:.2f}'.format(ultimo),xy=(indexUltimo,ultimo+2),fontsize=14,color='black',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])

  # Porcentagem Variação
  ax[i].annotate(variacao,xy=(indexUltimo,-50),fontsize=14,color='#d92027',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])
  
  # Tracinhos do Capeta
  # Tranformação do Timestamp em Periodo
  ax[i].plot([pd.Period(indexMin.ctime(), 'D'), pd.Period(indexUltimo.ctime(), 'D')], [min, min], color="#ff9234", linestyle="--", linewidth=2)
  ax[i].plot([pd.Period(indexUltimo.ctime(), 'D'), pd.Period(indexUltimo.ctime(), 'D')], [min, ultimo], color="#d92027", linestyle="--", linewidth=2)


  ## Fundo do Plot
  fig.patch.set_facecolor('#fff1e5')
  ax[i].set_facecolor('#fff1e5')
  fig.patch.set_visible('#fff1e5')
  # ax[i].spines['right'].set_visible(False)
  # ax[i].spines['left'].set_visible(False)
  # ax[i].spines['top'].set_visible(False)
  for i,regiao in  enumerate(['Norte','Nordeste','Centro-Oeste','Sudeste','Sul']):
    ax[i].set_title(regiao,fontsize=14,color='black',weight='bold',ha='center')
    ax[i].xaxis.grid(False)
    ax[i].set_facecolor('#fff1e5')
    ax[i].set_xlabel("")
    ax[i].arrow(0, 3, 5, -2.5, head_width = 0.5, 
          head_length = 0.5, fc ='g', ec ='g')
    ax[i].set_xticklabels(["","Ago","","","","","","Set"])
    ax[i].xaxis.set_tick_params(labelsize=12)
    ax[i].yaxis.set_tick_params(labelsize=12)
    if (i==0) or (i==4):
      ax[i].yaxis.tick_right()
    else:
      ax[i].set_yticklabels([])

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(32,8))
#Colocando cor na area de algo


estados = ["Pará", "Maranhão", "Mato Grosso","Minas Gerais","Paraná"]

df_todos_estados = agrupar(df,[df.index,df.estado])

for i,estado in enumerate(estados):

  #Agrupando por Index e Estado
  df_destaque = agrupar(df,[df.index,df.estado],estado)
  
  for regiao in [norte,nordeste,centro_oeste,sudeste,sul]:
    df_regiao = agrupar(df,[df.index,df.estado],regiao)
  ##Colocando os outros Estados (Por Regiao) em Cinza
  df_regiao.plot(legend=False,color="grey",linewidth=1, alpha=0.4, ax=ax[i])


  ## Colocando o Estado em Destaque
  df_destaque.plot(legend=True, linewidth=4,zorder=3,color='#35d0ba', alpha=1,ax=ax[i])


  # Pegar os Mínimos
  indexMin, min = getMinimo(df_destaque.dropna())
  indexUltimo, ultimo = getUltimo(df_destaque.dropna())
  variacao = getVariacao(min,ultimo)

  # Bolinha no Minimo
  ax[i].scatter(indexMin,min,marker='o',color='#ff9234',linewidth=6,zorder=4)
  ax[i].annotate('{0:.2f}'.format(min),xy=(indexMin,min+1),fontsize=14,color='black',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])
  

  # Bolinha no ultimo
  ax[i].scatter(indexUltimo,ultimo,marker='o',color='#ff9234',linewidth=6,zorder=4)
  ax[i].annotate('{0:.2f}'.format(ultimo),xy=(indexUltimo,ultimo+2),fontsize=14,color='black',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])

  # Porcentagem Variação
  ax[i].annotate(variacao,xy=(indexUltimo,-50),fontsize=14,color='#d92027',weight='bold',ha='right',path_effects=[path_effects.withStroke(linewidth=4,foreground='w')])
  
  # Tracinhos do Capeta
  # Tranformação do Timestamp em Periodo
  ax[i].plot([pd.Period(indexMin.ctime(), 'D'), pd.Period(indexUltimo.ctime(), 'D')], [min, min], color="#ff9234", linestyle="--", linewidth=2)
  ax[i].plot([pd.Period(indexUltimo.ctime(), 'D'), pd.Period(indexUltimo.ctime(), 'D')], [min, ultimo], color="#d92027", linestyle="--", linewidth=2)


  ## Fundo do Plot
  fig.patch.set_facecolor('#fff1e5')
  ax[i].set_facecolor('#fff1e5')
  fig.patch.set_visible('#fff1e5')
  # ax[i].spines['right'].set_visible(False)
  # ax[i].spines['left'].set_visible(False)
  # ax[i].spines['top'].set_visible(False)
  for i,regiao in  enumerate(['Norte','Nordeste','Centro-Oeste','Sudeste','Sul']):
    ax[i].set_title(regiao,fontsize=14,color='black',weight='bold',ha='center')
    ax[i].xaxis.grid(False)
    ax[i].set_facecolor('#fff1e5')
    ax[i].set_xlabel("")
    ax[i].arrow(0, 3, 5, -2.5, head_width = 0.5, 
          head_length = 0.5, fc ='g', ec ='g')
    ax[i].set_xticklabels(["","Ago","","","","","","Set"])
    ax[i].xaxis.set_tick_params(labelsize=12)
    ax[i].yaxis.set_tick_params(labelsize=12)
    if (i==0) or (i==4):
      ax[i].yaxis.tick_right()
    else:
      ax[i].set_yticklabels([])